In [3]:
from datetime import datetime
import requests
import json
import pandas as pd
import numpy as np

In [80]:

# 현재 시간 불러오기
def weather_now():
    now = datetime.now()

    # 원하는 형식으로 변환
    formatted_time = now.strftime("%Y%m%d%H%M")

    print(formatted_time)
    yyyyMMdd =formatted_time[:8]
    HH = formatted_time[8:10]
    HH_00 = int(HH)*100



    url = 'http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getUltraSrtNcst'
    params = {
        'serviceKey': 'Jh8gMjcJB2/c87a9lWgb0/uxDLf0pk756QwV8p0Rcu5ySVrZZLV0Y43KoesUbUTPD/+ZDXBp/amEtbhM7ZYquw==',
        'pageNo': '1',
        'numOfRows': '1000',
        'dataType': 'JSON',
        'base_date': f'{yyyyMMdd}',
        'base_time': f'{HH_00}',
        'nx': '55', 
        'ny': '127'
    }

    response = requests.get(url, params=params)
    text_content = response.content.decode('utf-8')

    # JSON 문자열을 파싱하여 딕셔너리로 변환
    data = json.loads(text_content)

    # 필요한 데이터 추출
    items = data["response"]["body"]["items"]["item"]

    # 데이터 프레임으로 변환
    df = pd.DataFrame(items)

    # 데이터 프레임 출력
    RH = float(df[df['category']=='REH']['obsrValue'])
    TA = float(df[df['category']=='T1H']['obsrValue'])
    return RH,TA

def cal_water(kg, coffee, ex, ex_hour):
    if ex_hour > 0:
        print("운동 2시간 전에 물 섭취")
    ex_list = {'걷기': 0.9,'빨리걷기':1.2,'달리기': 2,
        '계단오르기': 1.6,'자전거' :1.8,'줄넘기': 2.6,'등산': 1.5,
        '수영': 2.0,'에어로빅': 1.5,'체조': 1,'테니스': 1.9,'스케이트': 1.8,
        '스키': 1.6,'권투': 2.4,'농구': 2.1,'배구': 1,'축구': 2.5}
    
    lost_weight_w = ex_list[ex]*(1/15)*ex_hour

    coffee_w =  coffee*2*300
    base_w = kg*30

    return (lost_weight_w + coffee_w +base_w)/200


def celsius_to_fahrenheit(celsius):
    return (celsius * 9/5) + 32

def heat_index(T, R):
    # 상수 정의
    c1 = -42.379
    c2 = 2.04901523
    c3 = 10.14333127
    c4 = -0.22475541
    c5 = -0.00683783
    c6 = -0.05481717
    c7 = 0.00122874
    c8 = 0.00085282
    c9 = -0.00000199
    
    HI = (c1 + (c2 * T) + (c3 * R) + (c4 * T * R) + 
          (c5 * T**2) + (c6 * R**2) + 
          (c7 * T**2 * R) + (c8 * T * R**2) + 
          (c9 * T**2 * R**2))
    return HI

def calculate_heat_index(celsius, humidity):
    fahrenheit = celsius_to_fahrenheit(celsius)
    hi_fahrenheit = heat_index(fahrenheit, humidity)
    hi_celsius = (hi_fahrenheit - 32) * 5/9
    return hi_celsius

# 예제 사용법
RH,TA = weather_now()

hi = calculate_heat_index(TA, RH)

if hi > 31:
    duration = "20 분"
elif hi > 35:
    duration = "15 분"
else:
    duration = None

print(f"체감 온도는 {hi:.2f}°C 입니다.\n{duration}")


202406282255
체감 온도는 20.48°C 입니다.None


In [81]:
cal_water(80, 2, "걷기", 30)

운동 2시간 전에 물 섭취


18.009